In [1]:
from __future__ import print_function, division
import numpy as np
from os.path import join, expanduser
import matplotlib.pyplot as plt
import yaml  # for pretty-printing dict
from neuralnilm.metrics import run_metrics, across_all_appliances
import pandas as pd

# sklearn evokes warnings from numpy
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/home/dk3810/workspace/python/nntools/lasagne/init.py:86: UserWarning: The uniform initializer no longer uses Glorot et al.'s approach to determine the bounds, but defaults to the range (-0.01, 0.01) instead. Please use the new GlorotUniform initializer to get the old behavior. GlorotUniform is now the default for all layers.
  warnings.warn("The uniform initializer no longer uses Glorot et al.'s "


In [28]:
TRAIN_HOUSES = {
    'microwave': (1, 2),
    'fridge': (1, 2, 4),
    'dish washer': (1, 2),
    'kettle': (1, 2, 4),
    'washing machine': (1, 5)
}

TEST_HOUSES = {
    'microwave': (5,),
    'fridge': (5,),
    'dish washer': (5,),
    'kettle': (5,),
    'washing machine': (2,)
}

APPLIANCES = TRAIN_HOUSES.keys()

ON_POWER_THRESHOLDS = {
    'microwave': 200,
    'fridge': 30,
    'dish washer': 10,
    'kettle': 2000,
    'washing machine': 20
}

HOUSES = [1, 2, 3, 4, 5]

METRICS = [
    'f1_score',
    'precision_score',
    'recall_score',
    'accuracy_score',
    'relative_error_in_total_energy',
    'total_energy_correctly_assigned',
    'mean_absolute_error'
]

ALGORITHMS = ['co', 'fhmm', 'ae', 'rectangles', 'rnn']

full_algorithm_names = [
    'Combinatorial Optimisation ', 'Factorial HMM', 'Autoencoder', 'Rectangles', 'LSTM']


ESTIMATES_PATH = expanduser(
    "~/PhD/experiments/neural_nilm/data_for_BuildSys2015/disag_estimates")
GROUND_TRUTH_PATH = expanduser(
    "~/PhD/experiments/neural_nilm/data_for_BuildSys2015/ground_truth_and_mains")

PLOT_PATH = expanduser("~/PhD/writing/papers/BuildSys_2015_Neural_NILM")

In [3]:
def load(architecture, building_i, appliance):
    # load estimates
    estimates_fname = "{}_building_{}_estimates_{}.csv".format(
        architecture, building_i, appliance)
    estimates_fname = join(ESTIMATES_PATH, estimates_fname)
    y_pred = np.loadtxt(estimates_fname, delimiter=',')

    # load ground truth
    y_true_fname = "building_{}_{}.csv".format(building_i, appliance.replace(' ', '_'))
    y_true_fname = join(GROUND_TRUTH_PATH, y_true_fname)
    y_true = np.loadtxt(y_true_fname, delimiter=',')

    # load mains
    mains_fname = "building_{}_mains.csv".format(building_i)
    mains_fname = join(GROUND_TRUTH_PATH, mains_fname)
    mains = np.loadtxt(mains_fname, delimiter=',')

    return y_true, y_pred, mains

In [4]:
def plot_all(y_true, y_pred, mains, title=None):
    fig, axes = plt.subplots(nrows=3, sharex=True)
    axes[0].plot(y_pred)
    axes[0].set_title('y_pred')
    axes[1].plot(y_true)
    axes[1].set_title('y_true')
    axes[2].plot(mains)
    axes[2].set_title('mains')
    if title:
        fig.set_title(title)
    plt.show()
    return fig, axes

In [29]:
# Run metrics
def calc_metrics(houses):
    scores = pd.Panel(
        np.NaN,
        items=APPLIANCES,
        major_axis=METRICS,
        minor_axis=ALGORITHMS
    )
    
    for appliance in APPLIANCES:
        houses_for_appliance = houses[appliance]
        on_power_threshold = ON_POWER_THRESHOLDS[appliance]
        for algo in ALGORITHMS:
            house_scores = pd.DataFrame(
                np.NaN, columns=METRICS, index=houses_for_appliance)
            for house_i in houses_for_appliance:
                y_true, y_pred, mains = load(algo, house_i, appliance)
                house_scores_dict = run_metrics(
                    y_true, y_pred, mains, on_power_threshold)
                house_scores_dict.pop('sum_abs_diff')
                house_scores.loc[house_i] = house_scores_dict
            scores[appliance, :, algo].update(house_scores.dropna().mean())
    
    scores['across all appliances'] = scores.mean(axis=0)
    return scores

In [30]:
test_houses_scores = calc_metrics(TEST_HOUSES)
train_houses_scores = calc_metrics(TRAIN_HOUSES)

In [44]:
APPLIANCE = 'dish washer'
test_houses_scores[APPLIANCE]

,co,fhmm,ae,rectangles,rnn
f1_score,0.111038,0.047780,0.301147,0.697507,0.077491
precision_score,0.060572,0.025108,0.177880,0.838849,0.040547
recall_score,0.665529,0.492491,0.980887,0.596928,0.872014
accuracy_score,0.638623,0.334306,0.845613,0.982442,0.295914
relative_error_in_total_energy,0.622641,0.746979,0.558242,-0.335521,0.865344
total_energy_correctly_assigned,0.939789,0.910207,0.964643,0.975001,0.862678
mean_absolute_error,73.556146,109.694792,43.193715,30.539340,167.758252


In [45]:
train_houses_scores[APPLIANCE]

,co,fhmm,ae,rectangles,rnn
f1_score,0.112032,0.078415,0.356627,0.670908,0.063205
precision_score,0.065125,0.041302,0.225228,0.877992,0.033287
recall_score,0.499706,0.778211,0.985613,0.549394,0.626205
accuracy_score,0.686090,0.368784,0.860627,0.982688,0.351462
relative_error_in_total_energy,0.281835,0.657594,0.441358,-0.592472,0.762904
total_energy_correctly_assigned,0.897931,0.845668,0.947723,0.956008,0.831665
mean_absolute_error,75.219464,111.018239,43.255669,29.995375,129.677987


In [46]:
y_true, y_pred, mains = load('rnn', 1, APPLIANCE)
plot_all(y_true, y_pred, mains)

(<matplotlib.figure.Figure at 0x7f3623702bd0>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x7f36242ce650>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x7f36249b3a50>], dtype=object))

In [66]:
# plot
COLOR = ['#5F7343', '#99A63C', '#FEC06A', '#F25430', '#E61924']
FONTSIZE = 10

def plot_scores(scores):
    appliances = list(scores.items)
    metrics = list(scores.major_axis)
    algorithms = list(scores.minor_axis)
    
    nrows = len(metrics)
    ncols = len(appliances)
    n_algorithms = len(algorithms)
    x = range(n_algorithms)
    
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharey='row', figsize=(8.1, 8.7))
    fig.patch.set_facecolor('white')
    for row_i, metric in enumerate(metrics):
        for col_i, appliance in enumerate(appliances):
            ax = axes[row_i, col_i]
            scores_for_algorithms = scores[appliance, metric]
            rects = ax.bar(
                x, scores_for_algorithms, color=COLOR, edgecolor=COLOR, zorder=3)

            # Numbers on the plot
            if row_i == 6:  # mean absolute error (watts)
                text_y = 90
                text_format = '{:3.0f}'
            elif row_i == 4:  # relative error in total energy
                text_y = 0
            else:
                text_y = 0.5
                text_format = '{:.2f}'

            # Draw text
            for i, rect in enumerate(rects):
                ax.text(
                    rect.get_x() + (1/6),
                    text_y,
                    text_format.format(scores_for_algorithms[i]),
                    va='center', rotation=90, fontsize=FONTSIZE)

            # Formatting
            ax.set_xticks([])
            ax.tick_params(direction='out')
            ax.yaxis.grid(
                b=True, which='major', color='white', linestyle='-', zorder=0)
            ax.patch.set_facecolor((0.85, 0.85, 0.85))

            if row_i == 4:  # relative error in total energy
                ax.set_ylim((-1, 1))

            for spine in ['top', 'right', 'left', 'bottom']:
                ax.spines[spine].set_visible(False)

            if row_i == 0:
                if appliance == 'across all appliances':
                    label = 'Across all\nappliances'
                else:
                    label = appliance.replace(' ', '\n')
                    label = label[0].capitalize() + label[1:]
                ax.set_title(label, fontsize=FONTSIZE)
            if col_i == 0:
                label = metric.replace('_', '\n')
                if label == 'mean\nabsolute\nerror':
                    label = label + '\n(watts)'
                elif label == 'total\nenergy\ncorrectly\nassigned':
                    label = 'prop. of\n' + label
                elif label == 'relative\nerror\nin\ntotal\nenergy':
                    label = 'relative\nerror in\ntotal\nenergy'
                label = label[0].capitalize() + label[1:]
                ylabel = ax.set_ylabel(label, fontsize=FONTSIZE)
                ylabel.set_rotation('horizontal')
                ylabel.set_verticalalignment('center')
                ylabel.set_horizontalalignment('center')
                ax.yaxis.labelpad = 25
                ax.tick_params(axis='y', left='on', right='off')
            else:
                ax.tick_params(axis='y', left='off', right='off')

    plt.subplots_adjust(hspace=0.3, top=0.96, bottom=0.07, left=0.13, right=0.99)
    plt.legend(rects, full_algorithm_names, ncol=n_algorithms, loc=(-7, -0.6),
               frameon=False, fontsize=FONTSIZE)
    return fig, axes

In [67]:
fig, axes = plot_scores(test_houses_scores)
#fig.suptitle('Unseen houses', fontsize=16)
plt.savefig(join(PLOT_PATH, 'unseen_houses.eps'))
plt.show()

In [68]:
fig, axes = plot_scores(train_houses_scores)
#fig.suptitle('Train houses', fontsize=16)
plt.savefig(join(PLOT_PATH, 'train_houses.eps'))
plt.show()